In [ ]:
!pip install category_encoders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import category_encoders as ce

root = '/content/drive/MyDrive/instacart-market-basket-analysis/'

In [ ]:
aisles = pd.read_csv(root + 'aisles.csv')

departments = pd.read_csv(root + 'departments.csv')

orders = pd.read_csv(root + 'orders.csv',
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})
order_products_prior = pd.read_csv(
    root + 'order_products__prior.csv',
    dtype={
        'order_id': 'Int32',  # Nullable integer dtype
        'product_id': 'Int32',
        'add_to_cart_order': 'Int16',
        'reordered': 'Int8'
    }
)

order_products_train = pd.read_csv(root + 'order_products__train.csv',
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

products = pd.read_csv(root + 'products.csv')

Preparing Data

In [ ]:
prior_df = order_products_prior.merge(orders, on ='order_id', how='inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86.0,16.0
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83.0,4.0
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104.0,13.0
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19.0,13.0
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17.0,13.0


Features creation
Calculating how many times a user buy the product

In [ ]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86.0,16.0,1.0
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83.0,4.0,1.0
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104.0,13.0,1.0
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19.0,13.0,1.0
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17.0,13.0,1.0


Product level features
(1) Product's average add-to-cart-order

(2) Total times the product was ordered

(3) Total times the product was reordered

(4) Reorder percentage of a product

(5) Total unique users of a product

(6) Is the product Organic?

(7) Percentage of users that buy the product second time

In [ ]:
agg_dict1 = {'add_to_cart_order' : {'mean_add_to_cart_order':'mean'},
           'reordered' : {'total_orders':'count', 'total_reorders':'sum', 'reorder_percentage':'mean'},
           'user_id': {'unique_users' :lambda x: x.nunique()},
           'user_buy_product_times': {'order_first_time_total_cnt' : lambda x: sum(x==1),
                                      'order_second_time_total_cnt' :lambda x: sum(x==2)},
           'product_name':{'is_organic': lambda x: 1 if 'Organic' in x else 0}}

In [ ]:
agg_dict1 = {
    'reordered': ['mean', 'sum']
}

prod_feats1 = prior_df.groupby('product_id').agg(agg_dict1)
prod_feats1.columns = ['prod_reordered_mean', 'prod_reordered_sum']
prod_feats1.reset_index(inplace=True)


In [ ]:
# Example assuming prior_df has 'product_id' and 'reordered'
# First-time orders: reordered == 0
# Second-time or more: reordered == 1

first_orders = prior_df[prior_df['reordered'] == 0].groupby('product_id').size().rename('order_first_time_total_cnt')
second_orders = prior_df[prior_df['reordered'] == 1].groupby('product_id').size().rename('order_second_time_total_cnt')

prod_feats1 = pd.concat([first_orders, second_orders], axis=1).fillna(0)

prod_feats1['second_time_percent'] = prod_feats1['order_second_time_total_cnt'] / prod_feats1['order_first_time_total_cnt']


Aisle and department features
(8) Reorder percentage, Total orders and reorders of a product aisle

(9) Mean and std of aisle add-to-cart-order

(10) Aisle unique users

In [ ]:
agg_dict2 = {'add_to_cart_order' : {'aisle_mean_add_to_cart_order':'mean',
                                   'aisle_std_add_to_cart_order':'std'},
           'reordered' : {'aisle_total_orders':'count', 'aisle_total_reorders':'sum', 'aisle_reorder_percentage':'mean'},
           'user_id': {'aisle_unique_users' :lambda x: x.nunique()}}

In [ ]:
aisle_feats = prior_df.groupby('aisle_id').agg(
    aisle_mean_add_to_cart_order=pd.NamedAgg(column='add_to_cart_order', aggfunc='mean'),
    aisle_std_add_to_cart_order=pd.NamedAgg(column='add_to_cart_order', aggfunc='std'),

    aisle_total_orders=pd.NamedAgg(column='reordered', aggfunc='count'),
    aisle_total_reorders=pd.NamedAgg(column='reordered', aggfunc='sum'),
    aisle_reorder_percentage=pd.NamedAgg(column='reordered', aggfunc='mean'),

    aisle_unique_users=pd.NamedAgg(column='user_id', aggfunc=lambda x: x.nunique())
)

aisle_feats.reset_index(inplace=True)



features

(10) Reorder percentage, Total orders and reorders of a product department

(11) Mean and std of department add-to-cart-order

(12) Department unique users

In [ ]:
agg_dict3 = {'add_to_cart_order' : {'department_mean_add_to_cart_order':'mean',
                                   'department_std_add_to_cart_order':'std'},
           'reordered' : {'department_total_orders':'count', 'department_total_reorders':'sum',
                          'department_reorder_percentage':'mean'},
           'user_id': {'department_unique_users' :lambda x: x.nunique()}}

In [ ]:
dpt_feats = prior_df.groupby('department_id').agg(
    department_mean_add_to_cart_order=pd.NamedAgg(column='add_to_cart_order', aggfunc='mean'),
    department_std_add_to_cart_order=pd.NamedAgg(column='add_to_cart_order', aggfunc='std'),

    department_total_orders=pd.NamedAgg(column='reordered', aggfunc='count'),
    department_total_reorders=pd.NamedAgg(column='reordered', aggfunc='sum'),
    department_reorder_percentage=pd.NamedAgg(column='reordered', aggfunc='mean'),

    department_unique_users=pd.NamedAgg(column='user_id', aggfunc=lambda x: x.nunique())
)

dpt_feats.reset_index(inplace=True)


features

(13) Binary encoding of aisle feature

(14) Binary encoding of department feature

In [ ]:
prod_feats1 = prod_feats1.merge(products, on = 'product_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisle_feats, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisles, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(dpt_feats, on = 'department_id', how = 'left')
prod_feats1 = prod_feats1.merge(departments, on = 'department_id', how = 'left')
prod_feats1.head()

,product_id,order_first_time_total_cnt,order_second_time_total_cnt,second_time_percent,product_name,aisle_id,department_id,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,323.0,472.0,1.461300,Chocolate Sandwich Cookies,61,19,9.287208,7.820526,101195,...,0.548397,36061,cookies cakes,9.191761,7.681034,1252447,719323,0.574334,144403,snacks
1,2,35.0,5.0,0.142857,All-Seasons Salt,104,13,9.973083,7.949225,91245,...,0.153104,45756,spices seasonings,9.581514,7.869214,811451,282131,0.347687,138915,pantry
2,3,31.0,81.0,2.612903,Robust Golden Unsweetened Oolong Tea,94,7,8.555624,7.915439,108199,...,0.525948,34687,tea,6.974485,6.7162,1164495,760722,0.653263,144648,beverages
3,4,70.0,71.0,1.014286,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,9.237562,7.473914,169341,...,0.557201,41090,frozen meals,9.003728,7.392699,969271,525355,0.54201,133681,frozen
4,5,1.0,6.0,6.000000,Green Chile Anytime Sauce,5,13,10.282027,8.245271,27313,...,0.279867,18119,marinades meat preparation,9.581514,7.869214,811451,282131,0.347687,138915,pantry


In [ ]:
prod_feats1.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
prod_feats1.head()

,product_id,order_first_time_total_cnt,order_second_time_total_cnt,second_time_percent,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,323.0,472.0,1.461300,9.287208,7.820526,101195,55495,0.548397,36061,cookies cakes,9.191761,7.681034,1252447,719323,0.574334,144403,snacks
1,2,35.0,5.0,0.142857,9.973083,7.949225,91245,13970,0.153104,45756,spices seasonings,9.581514,7.869214,811451,282131,0.347687,138915,pantry
2,3,31.0,81.0,2.612903,8.555624,7.915439,108199,56907,0.525948,34687,tea,6.974485,6.7162,1164495,760722,0.653263,144648,beverages
3,4,70.0,71.0,1.014286,9.237562,7.473914,169341,94357,0.557201,41090,frozen meals,9.003728,7.392699,969271,525355,0.54201,133681,frozen
4,5,1.0,6.0,6.000000,10.282027,8.245271,27313,7644,0.279867,18119,marinades meat preparation,9.581514,7.869214,811451,282131,0.347687,138915,pantry


In [ ]:
prod_feats1.shape


(49174, 18)

In [ ]:
prod_feats1.dtypes


,0
product_id,Int32
order_first_time_total_cnt,float64
order_second_time_total_cnt,float64
second_time_percent,float64
aisle_mean_add_to_cart_order,Float64
aisle_std_add_to_cart_order,Float64
aisle_total_orders,Int64
aisle_total_reorders,Int64
aisle_reorder_percentage,Float64
aisle_unique_users,int64


In [ ]:
encoder= ce.BinaryEncoder(cols=['aisle', 'department'],return_df=True)


In [ ]:
prod_feats1 = encoder.fit_transform(prod_feats1)
prod_feats1.head()

,product_id,order_first_time_total_cnt,order_second_time_total_cnt,second_time_percent,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_percentage,aisle_unique_users,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,323.0,472.0,1.461300,9.287208,7.820526,101195,55495,0.548397,36061,...,7.681034,1252447,719323,0.574334,144403,0,0,0,0,1
1,2,35.0,5.0,0.142857,9.973083,7.949225,91245,13970,0.153104,45756,...,7.869214,811451,282131,0.347687,138915,0,0,0,1,0
2,3,31.0,81.0,2.612903,8.555624,7.915439,108199,56907,0.525948,34687,...,6.7162,1164495,760722,0.653263,144648,0,0,0,1,1
3,4,70.0,71.0,1.014286,9.237562,7.473914,169341,94357,0.557201,41090,...,7.392699,969271,525355,0.54201,133681,0,0,1,0,0
4,5,1.0,6.0,6.000000,10.282027,8.245271,27313,7644,0.279867,18119,...,7.869214,811451,282131,0.347687,138915,0,0,0,1,0


In [ ]:
prod_feats1.shape

(49174, 29)

In [ ]:
prod_feats1.columns


Index(['product_id', 'order_first_time_total_cnt',
       'order_second_time_total_cnt', 'second_time_percent',
       'aisle_mean_add_to_cart_order', 'aisle_std_add_to_cart_order',
       'aisle_total_orders', 'aisle_total_reorders',
       'aisle_reorder_percentage', 'aisle_unique_users', 'aisle_0', 'aisle_1',
       'aisle_2', 'aisle_3', 'aisle_4', 'aisle_5', 'aisle_6', 'aisle_7',
       'department_mean_add_to_cart_order', 'department_std_add_to_cart_order',
       'department_total_orders', 'department_total_reorders',
       'department_reorder_percentage', 'department_unique_users',
       'department_0', 'department_1', 'department_2', 'department_3',
       'department_4'],
      dtype='object')

In [ ]:
prod_feats1.isnull().any().any()


False

In [ ]:
# free some memory
del aisle_feats, dpt_feats, aisles, departments
gc.collect()

60

In [ ]:
prior_df.isnull().any()


,0
order_id,False
product_id,True
add_to_cart_order,True
reordered,True
user_id,False
eval_set,False
order_number,False
order_dow,False
order_hour_of_day,False
days_since_prior_order,True


In [ ]:
# when no prior order, the value is null. Imputing as 0
prior_df.days_since_prior_order = prior_df.days_since_prior_order.fillna(0)

In [ ]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86.0,16.0,1.0
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83.0,4.0,1.0
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104.0,13.0,1.0
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19.0,13.0,1.0
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17.0,13.0,1.0


In [ ]:
agg_dict4 = {'order_dow': {'avg_dow':'mean', 'std_dow':'std'},
           'order_hour_of_day': {'avg_doh':'mean', 'std_doh':'std'},
           'days_since_prior_order': {'avg_since_order':'mean', 'std_since_order':'std'},
           'order_number': {'total_orders_by_user': lambda x: x.nunique()},
           'product_id': {'total_products_by_user': 'count',
                         'total_unique_product_by_user': lambda x: x.nunique()},
           'reordered': {'total_reorders_by_user':'sum',
                        'reorder_propotion_by_user':'mean'}}

In [ ]:
user_feats = prior_df.groupby('user_id').agg(
    avg_dow=pd.NamedAgg(column='order_dow', aggfunc='mean'),
    std_dow=pd.NamedAgg(column='order_dow', aggfunc='std'),

    avg_doh=pd.NamedAgg(column='order_hour_of_day', aggfunc='mean'),
    std_doh=pd.NamedAgg(column='order_hour_of_day', aggfunc='std'),

    avg_since_order=pd.NamedAgg(column='days_since_prior_order', aggfunc='mean'),
    std_since_order=pd.NamedAgg(column='days_since_prior_order', aggfunc='std'),

    total_orders_by_user=pd.NamedAgg(column='order_number', aggfunc=lambda x: x.nunique()),

    total_products_by_user=pd.NamedAgg(column='product_id', aggfunc='count'),
    total_unique_product_by_user=pd.NamedAgg(column='product_id', aggfunc=lambda x: x.nunique()),

    total_reorders_by_user=pd.NamedAgg(column='reordered', aggfunc='sum'),
    reorder_propotion_by_user=pd.NamedAgg(column='reordered', aggfunc='mean')
)

user_feats.reset_index(inplace=True)


features

(23) Average order size of a user

(24) User's mean of reordered items of all orders

In [ ]:
agg_dict5 = {
    'reordered': {
        'average_order_size': 'count',
        'reorder_in_order': 'mean'
    }
}


user_feats2 = prior_df.groupby(['user_id', 'order_number']).agg(
    average_order_size=pd.NamedAgg(column='reordered', aggfunc='count'),
    reorder_in_order=pd.NamedAgg(column='reordered', aggfunc='mean')
).reset_index()


In [ ]:
user_feats3 = user_feats2.groupby('user_id').agg({'average_order_size' : 'mean',
                                   'reorder_in_order':'mean'})
user_feats3 = user_feats3.reset_index()
user_feats3.head()

,user_id,average_order_size,reorder_in_order
0,1,6.0,0.741667
1,2,15.0,0.587457
2,3,7.2,0.633333
3,4,4.5,0.071429
4,5,12.0,0.666667


In [ ]:
user_feats = user_feats.merge(user_feats3, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order
0,1,2.888889,1.278275,12.500000,2.572479,23.833334,3.853951,3,18,10,13,0.722222,6.0,0.741667
1,2,1.733333,0.746528,10.755556,2.062567,16.900000,8.784627,6,90,55,52,0.577778,15.0,0.587457
2,3,1.694444,1.166667,16.305556,2.025825,9.111111,6.390966,5,36,19,19,0.527778,7.2,0.633333
3,4,4.222222,0.440959,14.555556,0.881917,19.666666,2.645751,2,9,9,1,0.111111,4.5,0.071429
4,5,1.000000,0.000000,18.000000,0.000000,19.000000,0.000000,1,12,12,8,0.666667,12.0,0.666667


features

(25) Percentage of reordered itmes in user's last three orders

(26) Total orders in user's last three orders

Last 3 orders of a user

In [ ]:
last_three_orders = user_feats2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders.head()

,user_id,level_1,order_number
0,1,2,7
1,1,1,5
2,1,0,3
3,2,8,14
4,2,7,11


In [ ]:
last_three_orders = user_feats2.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1
0,1,3,5,0.6,0
1,1,5,8,0.625,1
2,1,7,5,1.0,2
3,2,10,9,0.888889,6
4,2,11,15,0.8,7


In [ ]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank("dense", ascending=True)


In [ ]:
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['average_order_size', 'reorder_in_order']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,5.0,8.0,5.0,0.6,0.625,1.0
1,2,9.0,15.0,16.0,0.888889,0.8,0.625
2,3,6.0,5.0,6.0,0.833333,1.0,1.0
3,4,7.0,2.0,<NA>,0.142857,0.0,<NA>
4,5,12.0,<NA>,<NA>,0.666667,<NA>,<NA>


In [ ]:
user_feats = user_feats.merge(last_order_feats, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.888889,1.278275,12.500000,2.572479,23.833334,3.853951,3,18,10,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0
1,2,1.733333,0.746528,10.755556,2.062567,16.900000,8.784627,6,90,55,52,0.577778,15.0,0.587457,9.0,15.0,16.0,0.888889,0.8,0.625
2,3,1.694444,1.166667,16.305556,2.025825,9.111111,6.390966,5,36,19,19,0.527778,7.2,0.633333,6.0,5.0,6.0,0.833333,1.0,1.0
3,4,4.222222,0.440959,14.555556,0.881917,19.666666,2.645751,2,9,9,1,0.111111,4.5,0.071429,7.0,2.0,<NA>,0.142857,0.0,<NA>
4,5,1.000000,0.000000,18.000000,0.000000,19.000000,0.000000,1,12,12,8,0.666667,12.0,0.666667,12.0,<NA>,<NA>,0.666667,<NA>,<NA>


User and Product level features
(27) User's avg add-to-cart-order for a product

(28) User's avg days_since_prior_order for a product

(29) User's product total orders, reorders and reorders percentage

(30) User's order number when the product was bought last

In [ ]:
agg_dict6 = {'reordered': {'total_product_orders_by_user':'count',
                          'total_product_reorders_by_user':'sum',
                          'user_product_reorder_percentage': 'mean'},
            'add_to_cart_order': {'avg_add_to_cart_by_user':'mean'},
            'days_since_prior_order': {'avg_days_since_last_bought':'mean'},
            'order_number': {'last_ordered_in':'max'}}

In [ ]:
user_product_feats = prior_df.groupby(['user_id', 'product_id']).agg(
    total_product_orders_by_user=pd.NamedAgg(column='reordered', aggfunc='count'),
    total_product_reorders_by_user=pd.NamedAgg(column='reordered', aggfunc='sum'),
    user_product_reorder_percentage=pd.NamedAgg(column='reordered', aggfunc='mean'),

    avg_add_to_cart_by_user=pd.NamedAgg(column='add_to_cart_order', aggfunc='mean'),

    avg_days_since_last_bought=pd.NamedAgg(column='days_since_prior_order', aggfunc='mean'),

    last_ordered_in=pd.NamedAgg(column='order_number', aggfunc='max')
).reset_index()


features

(31) User's product purchase history of last three orders

In [ ]:
last_three_orders.head()


,user_id,order_number,average_order_size,reorder_in_order,level_1,rank
0,1,3,5,0.6,0,1.0
1,1,5,8,0.625,1,2.0
2,1,7,5,1.0,2,3.0
3,2,10,9,0.888889,6,1.0
4,2,11,15,0.8,7,2.0


In [ ]:
last_orders = prior_df.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times,average_order_size,reorder_in_order,level_1,rank
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86.0,16.0,1.0,9,0.666667,1366714,1.0
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83.0,4.0,1.0,9,0.666667,1366714,1.0
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104.0,13.0,1.0,9,0.666667,1366714,1.0
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19.0,13.0,1.0,9,0.666667,1366714,1.0
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17.0,13.0,1.0,9,0.666667,1366714,1.0


In [ ]:
last_orders['rank'] = last_orders.groupby(['user_id', 'product_id'])['order_number'].rank("dense", ascending=True)


In [ ]:
product_purchase_history = last_orders.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id', 'product_id', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1']
product_purchase_history.fillna(0, inplace = True)
product_purchase_history.head()

,user_id,product_id,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,10326,0.0,0.0,0.0
3,1,12427,1.0,1.0,1.0
4,1,13032,1.0,0.0,0.0


In [ ]:
user_product_feats = user_product_feats.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
user_product_feats.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,3,3,1.0,1.0,23.0,7,1.0,1.0,1.0
1,1,10258,3,3,1.0,2.666667,23.0,7,1.0,1.0,1.0
2,1,10326,1,0,0.0,5.0,28.0,5,0.0,0.0,0.0
3,1,12427,3,3,1.0,2.333333,23.0,7,1.0,1.0,1.0
4,1,13032,1,1,1.0,5.0,20.0,7,1.0,0.0,0.0


In [ ]:
user_product_feats.isnull().sum()


,0
user_id,0
product_id,0
total_product_orders_by_user,0
total_product_reorders_by_user,0
user_product_reorder_percentage,0
avg_add_to_cart_by_user,0
avg_days_since_last_bought,0
last_ordered_in,0
is_reorder_3,3563580
is_reorder_2,3563580


In [ ]:
user_product_feats.fillna(0, inplace = True)


Saving all features

In [ ]:
prod_feats1.to_pickle(root + 'product_features.pkl')
user_feats.to_pickle(root +'user_features.pkl')
user_product_feats.to_pickle(root +'user_product_features.pkl')

In [ ]:
df = pd.read_pickle(root +'product_features.pkl')
df.head()

,product_id,order_first_time_total_cnt,order_second_time_total_cnt,second_time_percent,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_percentage,aisle_unique_users,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,323.0,472.0,1.461300,9.287208,7.820526,101195,55495,0.548397,36061,...,7.681034,1252447,719323,0.574334,144403,0,0,0,0,1
1,2,35.0,5.0,0.142857,9.973083,7.949225,91245,13970,0.153104,45756,...,7.869214,811451,282131,0.347687,138915,0,0,0,1,0
2,3,31.0,81.0,2.612903,8.555624,7.915439,108199,56907,0.525948,34687,...,6.7162,1164495,760722,0.653263,144648,0,0,0,1,1
3,4,70.0,71.0,1.014286,9.237562,7.473914,169341,94357,0.557201,41090,...,7.392699,969271,525355,0.54201,133681,0,0,1,0,0
4,5,1.0,6.0,6.000000,10.282027,8.245271,27313,7644,0.279867,18119,...,7.869214,811451,282131,0.347687,138915,0,0,0,1,0


In [ ]:
df = pd.read_pickle(root+'user_features.pkl')
df.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.888889,1.278275,12.500000,2.572479,23.833334,3.853951,3,18,10,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0
1,2,1.733333,0.746528,10.755556,2.062567,16.900000,8.784627,6,90,55,52,0.577778,15.0,0.587457,9.0,15.0,16.0,0.888889,0.8,0.625
2,3,1.694444,1.166667,16.305556,2.025825,9.111111,6.390966,5,36,19,19,0.527778,7.2,0.633333,6.0,5.0,6.0,0.833333,1.0,1.0
3,4,4.222222,0.440959,14.555556,0.881917,19.666666,2.645751,2,9,9,1,0.111111,4.5,0.071429,7.0,2.0,<NA>,0.142857,0.0,<NA>
4,5,1.000000,0.000000,18.000000,0.000000,19.000000,0.000000,1,12,12,8,0.666667,12.0,0.666667,12.0,<NA>,<NA>,0.666667,<NA>,<NA>


In [ ]:
df = pd.read_pickle(root + 'user_product_features.pkl')
df.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,3,3,1.0,1.0,23.0,7,1.0,1.0,1.0
1,1,10258,3,3,1.0,2.666667,23.0,7,1.0,1.0,1.0
2,1,10326,1,0,0.0,5.0,28.0,5,0.0,0.0,0.0
3,1,12427,3,3,1.0,2.333333,23.0,7,1.0,1.0,1.0
4,1,13032,1,1,1.0,5.0,20.0,7,1.0,0.0,0.0
